Use Python to fetch country debt profiles. Use a horizontal bar chart, the y-axis is the list of all countries in the world (for example, Japan, Korea, Philippines, China, and more), and the x-axis is the total debt in $.  Make a ranking label on the outside and the right part of the bar, followed by the number of debt$ label

In [ ]:
import wbgapi as wb
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------
# 1) Select the debt indicator
# -------------------------
# Total external debt stocks (current USD)
series_code = "DT.DOD.DECT.CD"

# -------------------------
# 2) Fetch latest available data for all countries
# -------------------------
data = wb.data.fetch(series_code, economy="all", time=2024)  # latest year available
df = pd.DataFrame(data).reset_index()

# -------------------------
# 3) Inspect columns
# -------------------------
print("Columns returned:", df.columns)

# -------------------------
# 4) Detect debt column automatically
# -------------------------
# Usually the third column is the debt value
debt_col = [col for col in df.columns if col not in ["economy", "time"]][0]

# Rename columns
df = df.rename(columns={"economy": "Country", debt_col: "DebtUSD"})

# Drop missing debt values
df = df.dropna(subset=["DebtUSD"])

# -------------------------
# 5) Sort by debt and add ranking
# -------------------------
df = df.sort_values(by="DebtUSD", ascending=True)  # for horizontal bar chart
df["Rank"] = df["DebtUSD"].rank(method="first", ascending=False).astype(int)

# -------------------------
# 6) Plot horizontal bar chart
# -------------------------
plt.figure(figsize=(14, max(8, len(df)/3)))  # dynamic height
bars = plt.barh(df["Country"], df["DebtUSD"], color="steelblue")

plt.xlabel("Government Debt (USD)")
plt.title("National Government Debt by Country (Latest Available)")

# Add ranking and human-readable debt labels
for bar, rank, debt in zip(bars, df["Rank"], df["DebtUSD"]):
    width = bar.get_width()
    if debt >= 1e12:
        label = f"{debt/1e12:,.2f}T"
    elif debt >= 1e9:
        label = f"{debt/1e9:,.2f}B"
    else:
        label = f"{debt:,.0f}"
    plt.text(
        width * 1.01, 
        bar.get_y() + bar.get_height()/2, 
        f"#{rank} {label}", 
        va='center', fontsize=8
    )

plt.tight_layout()
plt.show()
